In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
data  = pd.read_csv("/content/drive/MyDrive/kdag_intra.csv")
data.head()

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


In [3]:
data.columns

Index(['isic_id', 'target', 'patient_id', 'age_approx', 'sex',
       'anatom_site_general', 'clin_size_long_diam_mm', 'image_type',
       'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
       'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
       'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'attribution', 'copyright_license', 'lesion_id',
       'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
       'mel_mitotic_index', '

In [33]:
data["age_approx"].fillna(data["age_approx"].median(),inplace=True)

In [32]:
data["sex"].fillna(data["sex"].mode()[0],inplace=True)
data["sex"] = data["sex"].map({"male":0 ,"female" : 1})

In [34]:
top_3_anatom_sites = data["anatom_site_general"].value_counts().index[:3].tolist()
data["anatom_site_general"] = data["anatom_site_general"].apply(lambda x: np.random.choice(top_3_anatom_sites) if pd.isna(x) else x)

In [61]:
# since the tbp_lv_location is a more elabore version of the simple
data.drop("tbp_lv_location_simple",axis =1,inplace=True)

In [35]:
data.isna().sum()

,0
isic_id,0
target,0
patient_id,0
age_approx,0
sex,0
anatom_site_general,0
clin_size_long_diam_mm,0
image_type,0
tbp_tile_type,0
tbp_lv_A,0


In [36]:
data[data["target"]==1].isna().sum()

,0
isic_id,0
target,0
patient_id,0
age_approx,0
sex,0
anatom_site_general,0
clin_size_long_diam_mm,0
image_type,0
tbp_tile_type,0
tbp_lv_A,0


In [38]:
data_lesion_id_na = data[data["lesion_id"].notna()]
data_lesion_id_na

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
1,ISIC_0015845,0,IP_8170065,60.0,0,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
49,ISIC_0073301,0,IP_2536274,40.0,1,posterior torso,9.59,TBP tile: close-up,3D: XP,24.237019,...,IL_3854228,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.886191
76,ISIC_0073989,0,IP_1307115,50.0,0,upper extremity,6.39,TBP tile: close-up,3D: white,19.600317,...,IL_9898853,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.997449
93,ISIC_0074611,0,IP_6164902,40.0,1,lower extremity,3.31,TBP tile: close-up,3D: XP,18.007100,...,IL_2988728,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000
107,ISIC_0075969,0,IP_5117525,40.0,1,posterior torso,9.05,TBP tile: close-up,3D: XP,21.315824,...,IL_6219432,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400997,ISIC_9998638,0,IP_2097401,40.0,0,lower extremity,3.90,TBP tile: close-up,3D: XP,16.600171,...,IL_0041380,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000
401019,ISIC_9999171,0,IP_7502403,50.0,1,anterior torso,5.51,TBP tile: close-up,3D: white,12.549400,...,IL_3334800,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,69.111250
401028,ISIC_9999315,0,IP_5337939,50.0,1,lower extremity,6.46,TBP tile: close-up,3D: XP,17.290400,...,IL_9124999,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999880
401054,ISIC_9999937,0,IP_1140263,70.0,0,anterior torso,6.80,TBP tile: close-up,3D: XP,22.574335,...,IL_9520694,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999988


# metadata categorization

In [62]:
target = data[['target','iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5','mel_mitotic_index','mel_thick_mm']]
patient = data[['patient_id','age_approx','sex','anatom_site_general']]
tbp_basic = data[['lesion_id','tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio','tbp_lv_location',
                  'tbp_lv_perimeterMM','tbp_lv_color_std_mean','tbp_lv_radial_color_std_max',
                  'clin_size_long_diam_mm','tbp_lv_norm_border','tbp_lv_norm_color']]
tbp_extra = data[['isic_id','tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
                  'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
                  'tbp_lv_Lext', 'tbp_lv_deltaA', 'tbp_lv_deltaB','tbp_lv_deltaL',
                  'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm','tbp_lv_eccentricity',
                  'tbp_lv_minorAxisMM','tbp_lv_stdL', 'tbp_lv_stdLExt',
                  'tbp_lv_symm_2axis','tbp_lv_symm_2axis_angle','tbp_lv_x', 'tbp_lv_y','tbp_lv_z']]
others = data[["attribution","lesion_id"]]
confidence_scores = data[['tbp_lv_dnn_lesion_confidence','tbp_lv_nevi_confidence']]

## target encoding

In [25]:
target

,target,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm
0,0,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
401054,0,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN
401055,0,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN
401056,0,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN
401057,0,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
iddx_1_enc = pd.DataFrame(ohe.fit_transform(target[["iddx_1"]]).astype(int).toarray(),columns = ohe.categories_[0])
iddx_1_enc

,Benign,Indeterminate,Malignant
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
401054,1,0,0
401055,1,0,0
401056,1,0,0
401057,1,0,0


## patient data encoding

In [26]:
patient

,patient_id,age_approx,sex,anatom_site_general
0,IP_1235828,60.0,male,lower extremity
1,IP_8170065,60.0,male,head/neck
2,IP_6724798,60.0,male,posterior torso
3,IP_4111386,65.0,male,anterior torso
4,IP_8313778,55.0,male,anterior torso
...,...,...,...,...
401054,IP_1140263,70.0,male,anterior torso
401055,IP_5678181,60.0,male,posterior torso
401056,IP_0076153,65.0,female,anterior torso
401057,IP_5231513,30.0,female,anterior torso


## tbp_lv_basic encoding

In [63]:
from sklearn.preprocessing import OneHotEncoder
location_ohe = OneHotEncoder()
tbp_lv_location = pd.DataFrame(location_ohe.fit_transform(tbp_basic[["tbp_lv_location"]]).astype(int).toarray(),columns = location_ohe.categories_[0])
tbp_basic.drop("tbp_lv_location",axis=1,inplace=True)
tbp_basic = pd.concat([tbp_basic,tbp_lv_location],axis=1)
tbp_basic

,lesion_id,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_perimeterMM,tbp_lv_color_std_mean,tbp_lv_radial_color_std_max,clin_size_long_diam_mm,tbp_lv_norm_border,tbp_lv_norm_color,Head & Neck,...,Right Leg - Lower,Right Leg - Upper,Torso Back,Torso Back Bottom Third,Torso Back Middle Third,Torso Back Top Third,Torso Front,Torso Front Bottom Half,Torso Front Top Half,Unknown
0,NaN,3.152561,27.476170,9.307003,0.000000,0.000000,3.04,7.091360,0.000000,0,...,0,1,0,0,0,0,0,0,0,0
1,IL_6727506,0.919497,12.235290,3.354148,0.000000,0.000000,1.10,2.116402,0.000000,1,...,0,0,0,0,0,0,0,0,0,0
2,NaN,3.265153,24.184620,8.886309,0.000000,0.000000,3.40,4.798335,0.000000,0,...,0,0,0,0,0,1,0,0,0,0
3,NaN,6.079940,14.889242,9.514499,0.514520,0.664690,3.22,1.975874,1.771705,0,...,0,0,0,0,0,0,0,0,1,0
4,NaN,2.101708,19.902560,6.467562,0.000000,0.000000,2.73,3.658854,0.000000,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,IL_9520694,22.893601,17.842449,20.210836,2.579861,2.328066,6.80,3.101811,7.348126,0,...,0,0,0,0,0,0,0,0,1,0
401055,NaN,4.785138,18.231470,9.340242,0.812333,1.048147,3.11,4.439972,2.795246,0,...,0,0,0,0,0,1,0,0,0,0
401056,IL_9852274,2.758491,13.050010,5.999862,0.498197,0.607554,2.05,1.444499,1.660411,0,...,0,0,0,0,0,0,0,0,1,0
401057,NaN,6.023644,13.787630,9.113276,1.317875,1.078204,2.80,1.421536,3.583966,0,...,0,0,0,0,0,0,0,1,0,0


## tbp_extra encoding

In [55]:
tbp_extra["tbp_tile_type"] = tbp_extra["tbp_tile_type"].map({"3D: white" : 0, "3D: XP" : 1})
tbp_extra

,isic_id,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,...,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_minorAxisMM,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z
0,ISIC_0015670,0,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,55.828924,...,5.784302,0.901302,1.543016,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948
1,ISIC_0015845,0,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,44.064040,...,4.987244,0.639885,0.821918,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500
2,ISIC_0015864,1,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,62.909730,...,6.290359,0.932147,1.194905,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900
3,ISIC_0015902,1,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,60.060388,...,6.400196,0.654458,2.481328,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802
4,ISIC_0024200,0,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,52.041180,...,6.531302,0.946448,0.929916,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,1,22.574335,14.944666,27.663259,26.767135,35.705132,30.656525,50.784168,60.824438,...,13.498163,0.744495,4.877998,7.054819,2.169398,0.288920,100,147.187256,1182.317505,122.652588
401055,ISIC_9999951,0,19.977640,16.026870,34.158840,31.011870,39.571870,34.908400,59.678970,62.670170,...,5.681591,0.719667,2.157864,1.879502,2.910780,0.460000,25,52.349740,1393.187000,127.261700
401056,ISIC_9999960,1,17.332567,12.364397,29.845326,26.500073,34.513206,29.242644,59.854275,64.987196,...,7.842607,0.649815,1.595713,1.702824,2.205272,0.183099,40,54.622246,1065.263672,-106.833740
401057,ISIC_9999964,1,22.288570,9.564721,28.431200,27.012250,36.126360,28.655640,51.905420,70.501540,...,8.515440,0.492929,2.547945,3.680175,1.957157,0.161850,140,-9.861557,877.527000,-76.982120


## confidence scores

In [16]:
confidence_scores

,tbp_lv_dnn_lesion_confidence,tbp_lv_nevi_confidence
0,97.517282,2.628592e-03
1,3.141455,1.334303e-07
2,99.804040,2.959177e-04
3,99.989998,2.198945e+01
4,70.442510,1.378832e-03
...,...,...
401054,99.999988,9.936233e+01
401055,99.999820,2.311562e-01
401056,99.999416,5.994798e+01
401057,100.000000,9.931933e+01


# null values managed

In [64]:
tbp_basic[tbp_basic["lesion_id"].notna()]

,lesion_id,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_perimeterMM,tbp_lv_color_std_mean,tbp_lv_radial_color_std_max,clin_size_long_diam_mm,tbp_lv_norm_border,tbp_lv_norm_color,Head & Neck,...,Right Leg - Lower,Right Leg - Upper,Torso Back,Torso Back Bottom Third,Torso Back Middle Third,Torso Back Top Third,Torso Front,Torso Front Bottom Half,Torso Front Top Half,Unknown
1,IL_6727506,0.919497,12.235290,3.354148,0.000000,0.000000,1.10,2.116402,0.000000,1,...,0,0,0,0,0,0,0,0,0,0
49,IL_3854228,31.975980,22.662001,26.919133,3.947276,3.215515,9.59,2.910906,9.913684,0,...,0,0,0,0,1,0,0,0,0,0
76,IL_9898853,28.617001,18.052941,22.729299,1.522630,1.807261,6.39,2.187051,4.998895,0,...,0,0,0,0,0,0,0,0,0,0
93,IL_2988728,5.948583,14.665700,9.340242,1.255051,1.006729,3.31,1.791452,3.382435,0,...,0,1,0,0,0,0,0,0,0,0
107,IL_6219432,46.162507,16.830977,27.874004,3.170209,3.187016,9.05,1.645372,9.528097,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400997,IL_0041380,10.602364,14.928672,12.580907,2.215048,1.429700,3.90,1.716784,5.439301,0,...,0,0,0,0,0,0,0,0,0,0
401019,IL_3334800,19.703510,15.091750,17.244140,5.393046,5.922404,5.51,1.109157,10.000000,0,...,0,0,0,0,0,0,0,0,1,0
401028,IL_9124999,23.531620,18.823870,21.046520,1.714663,1.654323,6.46,3.158871,5.047340,0,...,0,0,0,0,0,0,0,0,0,0
401054,IL_9520694,22.893601,17.842449,20.210836,2.579861,2.328066,6.80,3.101811,7.348126,0,...,0,0,0,0,0,0,0,0,1,0


# data augmentation

# EDA

# feature engineering

# Undersampling

# image loading

# image pre processing

# image augmentation